# Importing packages

In [ ]:
import numpy as np
import mirdata
from pesto import load_model

# Benchmarking on MDB-stem-synth


In [ ]:
dataset = mirdata.initialize("mdb_stem_synth")
dataset.download()